In [ ]:
import ast
import numpy as np
import torch
import torch.nn.functional as F
from datasets import load_dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score, precision_score, recall_score
import optuna
import os

## 1. Загрузка и подготовка данных

In [ ]:
raw_dataset = load_dataset("csv", data_files={"data": "/kaggle/input/resume/resume_dataset.csv"})

def process_labels(example):
    example["labels"] = ast.literal_eval(example["labels"])
    example["labels"] = [float(x) for x in example["labels"]]
    return example

dataset = raw_dataset["data"].map(process_labels)
print("Dataset size:", len(dataset))

split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

## 2. Токенизация

In [ ]:
model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

tokenized_datasets = split_dataset.map(tokenize_function, batched=True)

## 3. Вычисление весов классов

In [ ]:
def compute_class_weights(dataset):
    total = len(dataset)
    num_labels = len(dataset[0]["labels"])
    label_sums = np.zeros(num_labels)
    for example in dataset:
        label_sums += np.array(example["labels"])
    epsilon = 1e-8
    pos_weight = (total - label_sums) / (label_sums + epsilon)
    return torch.tensor(pos_weight, dtype=torch.float)

pos_weight = compute_class_weights(tokenized_datasets["train"])
print("pos_weight:", pos_weight)

## 4. Инициализация модели

In [ ]:
def model_init():
    config = AutoConfig.from_pretrained(model_checkpoint, num_labels=34, problem_type="multi_label_classification")
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, config=config)

# Глобальная переменная порога (будет обновляться в Optuna)
THRESHOLD = 0.45

## 5. Функция вычисления метрик

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    predictions = (probs > THRESHOLD).astype(int)
    micro_f1 = f1_score(labels, predictions, average="micro", zero_division=0)
    macro_f1 = f1_score(labels, predictions, average="macro", zero_division=0)
    # Вычисляем F1 по заданной формуле (используем eval_precision и eval_recall)
    precision = precision_score(labels, predictions, average="micro", zero_division=0)
    recall = recall_score(labels, predictions, average="micro", zero_division=0)
    if precision + recall > 0:
        f1_custom = 2 * (precision * recall) / (precision + recall)
    else:
        f1_custom = 0
    return {
         "eval_micro_f1": micro_f1,
         "eval_macro_f1": macro_f1,
         "eval_f1": f1_custom,  # Основной оптимизируемый показатель
         "eval_precision": precision,
         "eval_recall": recall
    }

## 6. Кастомный Trainer (без сохранения промежуточных чекпойнтов)

In [ ]:
class MyTrainer(Trainer):
    def __init__(self, *args, pos_weight=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight
        
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        if "labels" in inputs:
            inputs["labels"] = inputs["labels"].float()
        outputs = model(**inputs)
        logits = outputs.logits
        pos_weight_device = self.pos_weight.to(logits.device) if self.pos_weight is not None else None
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight_device)
        loss = loss_fct(logits, inputs["labels"])
        return (loss, outputs) if return_outputs else loss

## 7. Оптимизация с помощью Optuna (1 trial, эпохи от 10 до 14)

In [ ]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1.5e-05, 2.5e-05)
    weight_decay = trial.suggest_loguniform("weight_decay", 0.001, 0.002)
    batch_size = trial.suggest_categorical("batch_size", [16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 10, 14)
    threshold = trial.suggest_uniform("threshold", 0.46, 0.48)
    
    global THRESHOLD
    THRESHOLD = threshold
    
    training_args_trial = TrainingArguments(
        output_dir="./results_trial",
        eval_strategy="epoch",
        save_strategy="no",  # Не сохраняем промежуточные чекпойнты
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        warmup_steps=500,
        logging_steps=10,
        fp16=True,
        report_to="none"
    )
    
    model = model_init()
    
    trainer_trial = MyTrainer(
        model=model,
        args=training_args_trial,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        pos_weight=pos_weight
    )
    
    trainer_trial.train()
    eval_results = trainer_trial.evaluate()
    f1_custom = eval_results["eval_f1"]
    print(f"Trial finished: f1: {f1_custom:.4f}, params: {trial.params}")
    return f1_custom

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

print("Best trial:")
best_trial = study.best_trial
print("  Value: ", best_trial.value)
print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

## 8. Обучение лучшей модели с найденными гиперпараметрами и сохранение только лучшей модели

In [ ]:
best_lr = best_trial.params["learning_rate"]
best_wd = best_trial.params["weight_decay"]
best_bs = best_trial.params["batch_size"]
best_epochs = best_trial.params["num_train_epochs"]
best_thresh = best_trial.params["threshold"]

THRESHOLD = best_thresh

training_args_best = TrainingArguments(
    output_dir="./best_model_results",
    eval_strategy="epoch",
    save_strategy="epoch",  # Сохраняем чекпойнт после каждой эпохи
    save_total_limit=1,     # Храним только лучший чекпойнт
    learning_rate=best_lr,
    per_device_train_batch_size=best_bs,
    per_device_eval_batch_size=best_bs,
    num_train_epochs=best_epochs,
    weight_decay=best_wd,
    warmup_steps=500,
    logging_steps=10,
    fp16=True,
    load_best_model_at_end=True,  # После обучения загружается лучшая модель по метрике eval_f1
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    report_to="none"
)

trainer_best = MyTrainer(
    model_init=model_init,
    args=training_args_best,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    pos_weight=pos_weight
)

trainer_best.train()
final_eval = trainer_best.evaluate()
print("Final evaluation results:", final_eval)

# Сохраняем только лучшую модель и токенизатор в папку "Rock"
best_model_save_path = "./Rock"
trainer_best.save_model(best_model_save_path)
tokenizer.save_pretrained(best_model_save_path)
print(f"Best model saved to {best_model_save_path}")

# Архивируем модель в zip-файл "Rock.zip"
!zip -r Rock.zip "{best_model_save_path}"
print("Rock.zip created. Download it from the Output tab in Kaggle.")

Generating data split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/727 [00:00<?, ? examples/s]

Dataset size: 727


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

Trial finished: f1: 0.6804, params: {'learning_rate': 1.6466692666350026e-05, 'weight_decay': 0.0012022180401906955, 'batch_size': 16, 'num_train_epochs': 14, 'threshold': 0.4791430511285427}
Best trial:
  Value:  0.6954938552571689
  Params: 
    learning_rate: 1.9768352923720516e-05
    weight_decay: 0.0013309036353077938
    batch_size: 16
    num_train_epochs: 12
    threshold: 0.46269254347612143


<ipython-input-1-e5ce33bc2818>:92: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MyTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel

Epoch,Training Loss,Validation Loss,Micro F1,Macro F1,F1,Precision,Recall
1,0.768900,0.813504,0.494814,0.363229,0.494814,0.373391,0.733267
2,0.822400,0.813217,0.496225,0.363229,0.496225,0.375000,0.733267
3,0.749800,0.815925,0.516815,0.380462,0.516815,0.395172,0.746653
4,0.773500,0.819981,0.548018,0.397034,0.548018,0.425096,0.770947
5,0.798300,0.816580,0.574476,0.410430,0.574476,0.452063,0.787804
6,0.754700,0.815677,0.566860,0.417941,0.566860,0.447376,0.773426
7,0.748700,0.819297,0.590204,0.441485,0.590204,0.457775,0.830441
8,0.759300,0.815938,0.666923,0.462913,0.666923,0.544371,0.860684
9,0.792800,0.809999,0.694073,0.489557,0.694073,0.580828,0.862172
10,0.754800,0.791652,0.691312,0.487775,0.691312,0.590112,0.834408


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

Final evaluation results: {'eval_micro_f1': 0.6940730393135103, 'eval_macro_f1': 0.4895570593968406, 'eval_f1': 0.6940730393135103, 'eval_precision': 0.5808283233132933, 'eval_recall': 0.8621715418939019, 'eval_loss': 0.8099992275238037, 'eval_runtime': 1.6517, 'eval_samples_per_second': 88.393, 'eval_steps_per_second': 3.027, 'epoch': 12.0}
Best model saved to ./Rock
  adding: Rock/ (stored 0%)
  adding: Rock/config.json (deflated 67%)
  adding: Rock/sentencepiece.bpe.model (deflated 49%)
  adding: Rock/training_args.bin (deflated 51%)
  adding: Rock/special_tokens_map.json (deflated 52%)
  adding: Rock/model.safetensors (deflated 30%)
  adding: Rock/tokenizer.json (deflated 76%)
  adding: Rock/tokenizer_config.json (deflated 76%)
Rock.zip created. Download it from the Output tab in Kaggle.


In [2]:
print("Current THRESHOLD:", THRESHOLD)

Current THRESHOLD: 0.46269254347612143
